In [1]:
from __future__ import division
import sys, re, json
from collections import defaultdict

"""
Evaluate a set of test alignments versus the gold set. 
"""


class ParseError(Exception):
  def __init__(self, value):
    self.value = value
    
  def __str__(self):
    return self.value

class FScore:
  "Compute F1-Score based on gold set and test set."

  def __init__(self):
    self.gold = 0
    self.test = 0
    self.correct = 0

  def increment(self, gold_set, test_set):
    "Add examples from sets."
    self.gold += len(gold_set)
    self.test += len(test_set)
    self.correct += len(gold_set & test_set)

  def fscore(self): 
    pr = self.precision() + self.recall()
    if pr == 0: return 0.0
    return (2 * self.precision() * self.recall()) / pr

  def precision(self): 
    if self.test == 0: return 0.0
    return self.correct / self.test

  def recall(self): 
    if self.gold == 0: return 0.0
    return self.correct / self.gold    

  @staticmethod
  def output_header():
    "Output a scoring header."
    print ("%10s  %10s  %10s  %10s   %10s"%(
      "Type", "Total", "Precision", "Recall", "F1-Score"))
    print ("===============================================================")

  def output_row(self, name):
    "Output a scoring row."
    print ("%10s        %4d     %0.3f        %0.3f        %0.3f"%(
      name, self.gold, self.precision(), self.recall(), self.fscore()))

class CorpusAlignment:
  "Read in the alignment."
  def __init__(self, handle):
    self.all_align = set()
    self.sents_align = {}

    for l in handle:
      t = l.strip().split()
      if len(t) != 3: 
        raise ParseError("Alignment must have three columns. %s"%l)
      try:
        sent = int(t[0])
        align = (int(t[1]), int(t[2]))
        self.all_align.add((sent, align))
      except:
        raise ParseError("Alignment line must be integers. %s"%l)

  @staticmethod
  def compute_fscore(align1, align2):
    fscore = FScore()
    fscore.increment(align1.all_align, align2.all_align)
    return fscore

def main(gold_alignment, test_alignment):
  align1 = CorpusAlignment(gold_alignment)
  align2 = CorpusAlignment(test_alignment)
  fscore = CorpusAlignment.compute_fscore(align1, align2)
  FScore.output_header()
  fscore.output_row("total")

Initialize t(f|e)

In [2]:
ine = open("corpus.en","r")
corpuse = []
for e in ine:
    e = e.split()
    corpuse.append(e)
corpusf = []   
inf = open("corpus.es","r")
for f in inf:
    f = f.split()
    corpusf.append(f)

In [3]:
print(len(corpuse),len(corpusf))

(5401, 5401)


In [4]:
t1 = {}
t1['_NULL_'] = {}
for f in corpusf:
    for w in f:
        if w not in t1['_NULL_']:
            t1['_NULL_'][w] = 0

for w in t1['_NULL_']:
    t1['_NULL_'][w] = 1.0/(1.0*len(t1['_NULL_']))

for i in range(0,len(corpuse)):
    e = corpuse[i]
    f = corpusf[i]
    for w in e:
        if w not in t1:
            t1[w] = {}
        for fw in f:
            if fw not in t1[w]:
                t1[w][fw] = 0

for k in t1:
    for w in t1[k]:
        t1[k][w] = 1/len(t1[k])

IBM1

In [8]:
def initc():
    c1 = {}
    c2 = {}
    c3 = {}
    c4 = {}
    for k in range(0,len(corpuse)):
        e = corpuse[k]
        f = corpusf[k]
        for i in range(0,len(f)):
            if '_NULL_' not in c1:
                c1['_NULL_'] = 0
            if '_NULL_' not in c2:
                c2['_NULL_'] = {}
            if f[i] not in c2['_NULL_']:
                c2['_NULL_'][f[i]] = 0
            if (i+1,len(e),len(f)) not in c3:
                c3[(i+1,len(e),len(f))] = 0
            if (i+1,len(e),len(f)) not in c4:
                c4[(i+1,len(e),len(f))] = {}
            if 0 not in c4[(i+1,len(e),len(f))]:
                c4[i+1,len(e),len(f)][0]=0
            for j in range(0,len(e)):
                if e[j] not in c1:
                    c1[e[j]] = 0
                if e[j] not in c2:
                    c2[e[j]] = {}
                if f[i] not in c2[e[j]]:
                    c2[e[j]][f[i]] = 0
                if (i+1,len(e),len(f)) not in c3:
                    c3[(i+1,len(e),len(f))] = 0
                if (i+1,len(e),len(f)) not in c4:
                    c4[(i+1,len(e),len(f))] = {}
                if (j+1) not in c4[(i+1,len(e),len(f))]:
                    c4[i+1,len(e),len(f)][j+1]=0
    return c1,c2,c3,c4

In [10]:
deve = open("dev.en","r")
devcorpuse = []
for e in deve:
    e = e.split()
    devcorpuse.append(e)
devcorpusf = []   
devf = open("dev.es","r")
for f in devf:
    f = f.split()
    devcorpusf.append(f)

In [11]:
print(len(devcorpuse),len(devcorpusf))

(200, 200)


In [9]:
def evaluate (index):
    file = "dev"+str(index)+".out"
    out1 = open(file,'w')
    for k in range(0,len(devcorpuse)):
        e = devcorpuse[k]
        f = devcorpusf[k]
        for i in range(0,len(f)):
            maxt = t1['_NULL_'][f[i]]
            maxj = 0
            for j in range(0,len(e)):
                if t1[e[j]][f[i]] > maxt:
                    maxt =  t1[e[j]][f[i]]
                    maxj = j+1
            out1.write(str(k+1)+' '+str(maxj)+' '+str(i+1)+'\n')
    out1.close()
    main(open("dev.key"), open(file)) 

In [12]:
for index in range(0,5):
    c1,c2,c3,c4 = initc()
    for k in range(0,len(corpuse)):
        e = corpuse[k]
        f = corpusf[k]
        for i in range(0,len(f)):
            s = t1['_NULL_'][f[i]]
            for w in e:
                s = s+ t1[w][f[i]]
            delta = 1.0*t1['_NULL_'][f[i]]/(1.0*s)
            c1['_NULL_'] = c1['_NULL_']+delta
            c2['_NULL_'][f[i]] = c2['_NULL_'][f[i]]+delta
            for j in range(0,len(e)):
                delta = 1.0*t1[e[j]][f[i]]/(1.0*s)
                c1[e[j]] = c1[e[j]]+delta
                c2[e[j]][f[i]] = c2[e[j]][f[i]]+delta
    for e in t1:
        for f in t1[e]:
            t1[e][f] = 1.0*c2[e][f]/(1.0*c1[e])
    evaluate(index)

      Type       Total   Precision      Recall     F1-Score
     total        5920     0.211        0.217        0.214
      Type       Total   Precision      Recall     F1-Score
     total        5920     0.375        0.387        0.380
      Type       Total   Precision      Recall     F1-Score
     total        5920     0.402        0.415        0.408
      Type       Total   Precision      Recall     F1-Score
     total        5920     0.410        0.423        0.416
      Type       Total   Precision      Recall     F1-Score
     total        5920     0.413        0.427        0.420


IBM2

In [13]:
q2 = {}
for k in range(0,len(corpuse)):
        e = corpuse[k]
        f = corpusf[k]
        for i in range(0,len(f)):
            if (i+1,len(e),len(f)) not in q2:
                q2[(i+1,len(e),len(f))] = {}
            if 0 not in q2[(i+1,len(e),len(f))]:
                q2[(i+1,len(e),len(f))][0]=1.0/((len(e)+1)*1.0)
            for j in range(0,len(e)):
                if (j+1) not in q2[(i+1,len(e),len(f))]:
                    q2[(i+1,len(e),len(f))][j+1]=1.0/((len(e)+1)*1.0)

In [15]:
def evaluateIBM2(index):
    file = "devIBM2"+str(index)+".out"
    out2 = open(file,'w')
    for k in range(0,len(devcorpuse)):
        e = devcorpuse[k]
        f = devcorpusf[k]
        for i in range(0,len(f)):
            maxqt = q2[(i+1,len(e),len(f))][0]*t2['_NULL_'][f[i]]
            maxj = 0
            for j in range(0,len(e)):
                if q2[(i+1,len(e),len(f))][j+1]*t2[e[j]][f[i]] > maxqt:
                    maxqt =  q2[(i+1,len(e),len(f))][j+1]*t2[e[j]][f[i]]
                    maxj = j+1
            out2.write(str(k+1)+' '+str(maxj)+' '+str(i+1)+'\n')
    out2.close()
    main(open("dev.key"), open(file)) 

In [16]:
import copy
t2 = copy.deepcopy(t1)
for index in range(0,5):
    c1,c2,c3,c4 = initc()
    for k in range(0,len(corpuse)):
        e = corpuse[k]
        f = corpusf[k]
        for i in range(0,len(f)):
            s = q2[(i+1,len(e),len(f))][0]*t2['_NULL_'][f[i]]
            for ind in range(0,len(e)):
                s = s+ q2[(i+1,len(e),len(f))][ind+1]*t2[e[ind]][f[i]]
                
            delta = (1.0*q2[(i+1,len(e),len(f))][0]*t2['_NULL_'][f[i]])/(1.0*s)
            c1['_NULL_'] = c1['_NULL_']+delta
            c2['_NULL_'][f[i]] = c2['_NULL_'][f[i]]+delta
            c3[(i+1,len(e),len(f))] = c3[(i+1,len(e),len(f))]+delta
            c4[(i+1,len(e),len(f))][0] = c4[(i+1,len(e),len(f))][0]+delta
            for j in range(0,len(e)):
                delta = (1.0*q2[(i+1,len(e),len(f))][j+1]*t2[e[j]][f[i]])/(1.0*s)
                c1[e[j]] = c1[e[j]]+delta
                c2[e[j]][f[i]] = c2[e[j]][f[i]]+delta
                c3[(i+1,len(e),len(f))] = c3[(i+1,len(e),len(f))]+delta
                c4[(i+1,len(e),len(f))][j+1] = c4[(i+1,len(e),len(f))][j+1]+delta
    for e in t2:
        for f in t2[e]:
            t2[e][f] = 1.0*c2[e][f]/(1.0*c1[e])
    for key in q2:
        q2[key][0] = c4[key][0]*1.0/(c3[key]*1.0)
        for j in range(0,key[1]):
            q2[key][j+1] = c4[key][j+1]*1.0/(c3[key]*1.0)
            
    evaluateIBM2(index)

      Type       Total   Precision      Recall     F1-Score
     total        5920     0.432        0.446        0.439
      Type       Total   Precision      Recall     F1-Score
     total        5920     0.435        0.449        0.442
      Type       Total   Precision      Recall     F1-Score
     total        5920     0.439        0.453        0.446
      Type       Total   Precision      Recall     F1-Score
     total        5920     0.439        0.453        0.446
      Type       Total   Precision      Recall     F1-Score
     total        5920     0.442        0.456        0.449


Examples

In [72]:
gold = open("dev.key",'r')
test = open("devIBM2.out",'r')
senf = {}
sentences = {}
tesentences = {}
for g in gold:
    gsl = g.split()
    if gsl[0] not in sentences:
        senf[gsl[0]] = 0
        sentences[gsl[0]] = [gsl[0]+' '+gsl[1]+' '+gsl[2]]
    else:
        sentences[gsl[0]].append(gsl[0]+' '+gsl[1]+' '+gsl[2])

for t in test:
    gsl = t.split()
    if gsl[0] not in tesentences:
        tesentences[gsl[0]] = [gsl[0]+' '+gsl[1]+' '+gsl[2]]
    else:
        tesentences[gsl[0]].append(gsl[0]+' '+gsl[1]+' '+gsl[2])


In [73]:
for k in sentences:
    align1 = CorpusAlignment(sentences[k])
    align2 = CorpusAlignment(tesentences[k])
    fscore = CorpusAlignment.compute_fscore(align1, align2)
    senf[k] = fscore.fscore()

In [75]:
import operator
sort = sorted(senf.items(), key=operator.itemgetter(1))

In [89]:
print(devcorpuse[45])
print(devcorpusf[45])

['10', '.', 'considerably', 'limits', 'the', 'present', 'derogations', 'for', 'cement', 'kilns', '.']
['10', '-', 'limita', 'considerablemente', 'las', 'exenciones', 'vigentes', 'para', 'las', 'instalaciones', 'en', 'cementeras', ';']


In [81]:
print(sentences['46'],tesentences['46'])

(['46 1 1', '46 3 4', '46 4 3', '46 5 5', '46 6 7', '46 7 6', '46 8 8', '46 9 12', '46 10 10', '46 11 13'], ['46 1 1', '46 10 2', '46 10 3', '46 3 4', '46 9 5', '46 9 6', '46 10 7', '46 10 8', '46 9 9', '46 9 10', '46 9 11', '46 10 12', '46 9 13'])


Bonus

In [24]:
t11 = {}
t11['_NULL_'] = {}
for e in corpuse:
    for w in e:
        if w not in t11['_NULL_']:
            t11['_NULL_'][w] = 0

for w in t11['_NULL_']:
    t11['_NULL_'][w] = 1.0/(1.0*len(t11['_NULL_']))

for i in range(0,len(corpuse)):
    e = corpuse[i]
    f = corpusf[i]
    for w in f:
        if w not in t11:
            t11[w] = {}
        for fw in e:
            if fw not in t11[w]:
                t11[w][fw] = 0

for k in t11:
    for w in t11[k]:
        t11[k][w] = 1/len(t11[k])

In [22]:
def initccc():
    c1 = {}
    c2 = {}
    c3 = {}
    c4 = {}
    for k in range(0,len(corpusf)):
        e = corpuse[k]
        f = corpusf[k]
        for i in range(0,len(e)):
            if '_NULL_' not in c1:
                c1['_NULL_'] = 0
            if '_NULL_' not in c2:
                c2['_NULL_'] = {}
            if e[i] not in c2['_NULL_']:
                c2['_NULL_'][e[i]] = 0
            if (i+1,len(f),len(e)) not in c3:
                c3[(i+1,len(f),len(e))] = 0
            if (i+1,len(f),len(e)) not in c4:
                c4[(i+1,len(f),len(e))] = {}
            if 0 not in c4[(i+1,len(f),len(e))]:
                c4[i+1,len(f),len(e)][0]=0
            for j in range(0,len(f)):
                if f[j] not in c1:
                    c1[f[j]] = 0
                if f[j] not in c2:
                    c2[f[j]] = {}
                if e[i] not in c2[f[j]]:
                    c2[f[j]][e[i]] = 0
                if (i+1,len(f),len(e)) not in c3:
                    c3[(i+1,len(f),len(e))] = 0
                if (i+1,len(f),len(e)) not in c4:
                    c4[(i+1,len(f),len(e))] = {}
                if (j+1) not in c4[(i+1,len(f),len(e))]:
                    c4[i+1,len(f),len(e)][j+1]=0
    return c1,c2,c3,c4

In [20]:
def evaluateee (index):
    file = "dev"+str(index)+".out"
    out1 = open(file,'w')
    for k in range(0,len(devcorpusf)):
        e = devcorpuse[k]
        f = devcorpusf[k]
        for i in range(0,len(e)):
            maxt = t11['_NULL_'][e[i]]
            maxj = 0
            for j in range(0,len(f)):
                if t11[f[j]][e[i]] > maxt:
                    maxt =  t11[f[j]][e[i]]
                    maxj = j+1
            out1.write(str(k+1)+' '+str(maxj)+' '+str(i+1)+'\n')
    out1.close()
    main(open("dev.key"), open(file)) 

In [25]:
for index in range(0,5):
    c1,c2,c3,c4 = initccc()
    for k in range(0,len(corpusf)):
        e = corpuse[k]
        f = corpusf[k]
        for i in range(0,len(e)):
            s = t11['_NULL_'][e[i]]
            for w in f:
                s = s+ t11[w][e[i]]
            delta = 1.0*t11['_NULL_'][e[i]]/(1.0*s)
            c1['_NULL_'] = c1['_NULL_']+delta
            c2['_NULL_'][e[i]] = c2['_NULL_'][e[i]]+delta
            for j in range(0,len(f)):
                delta = 1.0*t11[f[j]][e[i]]/(1.0*s)
                c1[f[j]] = c1[f[j]]+delta
                c2[f[j]][e[i]] = c2[f[j]][e[i]]+delta
    for f in t11:
        for e in t11[f]:
            t11[f][e] = 1.0*c2[f][e]/(1.0*c1[f])
    evaluateee(index)

      Type       Total   Precision      Recall     F1-Score
     total        5920     0.050        0.049        0.050
      Type       Total   Precision      Recall     F1-Score
     total        5920     0.095        0.092        0.093
      Type       Total   Precision      Recall     F1-Score
     total        5920     0.100        0.098        0.099
      Type       Total   Precision      Recall     F1-Score
     total        5920     0.101        0.098        0.099
      Type       Total   Precision      Recall     F1-Score
     total        5920     0.101        0.099        0.100


In [26]:
q22 = {}
for k in range(0,len(corpuse)):
        e = corpuse[k]
        f = corpusf[k]
        for i in range(0,len(e)):
            if (i+1,len(f),len(e)) not in q22:
                q22[(i+1,len(f),len(e))] = {}
            if 0 not in q22[(i+1,len(f),len(e))]:
                q22[(i+1,len(f),len(e))][0]=1.0/((len(f)+1)*1.0)
            for j in range(0,len(f)):
                if (j+1) not in q22[(i+1,len(f),len(e))]:
                    q22[(i+1,len(f),len(e))][j+1]=1.0/((len(f)+1)*1.0)

In [32]:
def evaluateeeIBM2(index):
    file = "devIBM2"+str(index)+".out"
    out22 = open(file,'w')
    for k in range(0,len(devcorpusf)):
        e = devcorpuse[k]
        f = devcorpusf[k]
        for i in range(0,len(e)):
            maxqt = q22[(i+1,len(f),len(e))][0]*t22['_NULL_'][e[i]]
            maxj = 0
            for j in range(0,len(f)):
                if q22[(i+1,len(f),len(e))][j+1]*t22[f[j]][e[i]] > maxqt:
                    maxqt =  q22[(i+1,len(f),len(e))][j+1]*t22[f[j]][e[i]]
                    maxj = j+1
            out22.write(str(k+1)+' '+str(maxj)+' '+str(i+1)+'\n')
    out22.close()
    main(open("dev.key"), open(file)) 

In [33]:
import copy
t22 = copy.deepcopy(t11)
for index in range(0,5):
    c1,c2,c3,c4 = initccc()
    for k in range(0,len(corpuse)):
        e = corpuse[k]
        f = corpusf[k]
        for i in range(0,len(e)):
            s = q22[(i+1,len(f),len(e))][0]*t22['_NULL_'][e[i]]
            for ind in range(0,len(f)):
                s = s+ q22[(i+1,len(f),len(e))][ind+1]*t22[f[ind]][e[i]]
                
            delta = (1.0*q22[(i+1,len(f),len(e))][0]*t22['_NULL_'][e[i]])/(1.0*s)
            c1['_NULL_'] = c1['_NULL_']+delta
            c2['_NULL_'][e[i]] = c2['_NULL_'][e[i]]+delta
            c3[(i+1,len(f),len(e))] = c3[(i+1,len(f),len(e))]+delta
            c4[(i+1,len(f),len(e))][0] = c4[(i+1,len(f),len(e))][0]+delta
            for j in range(0,len(f)):
                delta = (1.0*q22[(i+1,len(f),len(e))][j+1]*t22[f[j]][e[i]])/(1.0*s)
                c1[f[j]] = c1[f[j]]+delta
                c2[f[j]][e[i]] = c2[f[j]][e[i]]+delta
                c3[(i+1,len(f),len(e))] = c3[(i+1,len(f),len(e))]+delta
                c4[(i+1,len(f),len(e))][j+1] = c4[(i+1,len(f),len(e))][j+1]+delta
    for f in t22:
        for e in t22[f]:
            t22[f][e] = 1.0*c2[f][e]/(1.0*c1[f])
    for key in q22:
        q22[key][0] = c4[key][0]*1.0/(c3[key]*1.0)
        for j in range(0,key[1]):
            q22[key][j+1] = c4[key][j+1]*1.0/(c3[key]*1.0)
            
    evaluateeeIBM2(index)

      Type       Total   Precision      Recall     F1-Score
     total        5920     0.112        0.109        0.111
      Type       Total   Precision      Recall     F1-Score
     total        5920     0.113        0.111        0.112
      Type       Total   Precision      Recall     F1-Score
     total        5920     0.112        0.109        0.110
      Type       Total   Precision      Recall     F1-Score
     total        5920     0.113        0.110        0.112
      Type       Total   Precision      Recall     F1-Score
     total        5920     0.114        0.111        0.112
